In [82]:
import pandas as pd
from pandasql import sqldf

In [83]:
#Utilizaremos essa função para rodar a consulta com a query 'q'
pysqldf = lambda q: sqldf(q, globals())

In [84]:
#Carregamos as bases que serão utilziadas
df_VRA = pd.read_csv('./airlines_VRA.csv', dtype={'icao_empresa_aerea':'str','numero_voo': 'str','codigo_autorizacao':'str'})
df_AIR_CIA = pd.read_csv('./airlines_AIR_CIA.csv')
df_aerodromos = pd.read_csv('./airlines_aerodromos_VRA.csv')

In [86]:
q = """WITH tabela_partida AS (SELECT icao_aerodromo_origem,
               icao_empresa_aerea,
               Count(*) AS qntd_partidas
        FROM   df_VRA
        GROUP  BY  icao_aerodromo_origem, icao_empresa_aerea),
        
        tabela_chegada AS (SELECT icao_aerodromo_destino,
               icao_empresa_aerea,
               Count(*) AS qntd_chegadas
        FROM   df_VRA
        GROUP  BY  icao_aerodromo_destino, icao_empresa_aerea),
        
        tabela_rotas_saida AS (SELECT icao_empresa_aerea,
               icao_aerodromo_destino,
               COUNT(DISTINCT icao_aerodromo_origem) as qntd_rotas
        FROM   df_VRA
        GROUP  BY icao_empresa_aerea, icao_aerodromo_destino),
        
        tabela_rotas_chegada AS (SELECT icao_empresa_aerea,
               icao_aerodromo_origem,
               COUNT(DISTINCT icao_aerodromo_destino) as qntd_rotas
        FROM   df_VRA
        GROUP  BY icao_empresa_aerea, icao_aerodromo_origem),
        
        tabela_atuacoes AS (SELECT R.icao_aerodromo_origem AS icao_aerodromo, R.icao_empresa_aerea, R.qntd_partidas + S.qntd_chegadas AS atuacao
        FROM tabela_partida R INNER JOIN tabela_chegada S
        ON R.icao_aerodromo_origem = S.icao_aerodromo_destino 
            AND R.icao_empresa_aerea = S.icao_empresa_aerea
        ORDER BY icao_aerodromo),
        
        tabela_pousos_decolagens AS (SELECT icao_aerodromo,
            icao_empresa_aerea,
            total_pousos_decolagens
        FROM
            (SELECT icao_aerodromo,
                icao_empresa_aerea,
                atuacao as total_pousos_decolagens,
                Rank() OVER (PARTITION BY icao_aerodromo ORDER BY atuacao DESC) AS rank_atuacao
            FROM tabela_atuacoes)
        WHERE rank_atuacao = 1),
        
        tabela_nome_aeroporto AS (SELECT S.name as nome_aeroporto, R.icao_aerodromo,  R.icao_empresa_aerea,  R.total_pousos_decolagens
        FROM tabela_pousos_decolagens R INNER JOIN df_aerodromos S 
        ON R.icao_aerodromo = S.icao),
        
        tabela_razao_social AS (SELECT R.nome_aeroporto, R.icao_aerodromo,  R.icao_empresa_aerea, S.razao_social,  R.total_pousos_decolagens
        FROM tabela_nome_aeroporto R LEFT JOIN df_AIR_CIA S 
        ON R.icao_empresa_aerea = S.icao), 
        
        tabela_quantidade_rotas_saida AS (SELECT R.nome_aeroporto, R.icao_aerodromo,  R.icao_empresa_aerea, R.razao_social,  R.total_pousos_decolagens, S.qntd_rotas as rotas_saida
        FROM tabela_razao_social R LEFT JOIN tabela_rotas_saida S 
        ON R.icao_empresa_aerea = S.icao_empresa_aerea AND R.icao_aerodromo = S.icao_aerodromo_destino)
        
        SELECT R.nome_aeroporto, R.icao_aerodromo,  R.icao_empresa_aerea, R.razao_social,  R.rotas_saida, S.qntd_rotas as rotas_chegada, R.total_pousos_decolagens
        FROM tabela_quantidade_rotas_saida R LEFT JOIN tabela_rotas_chegada S 
        ON R.icao_empresa_aerea = S.icao_empresa_aerea AND R.icao_aerodromo = S.icao_aerodromo_origem;"""


names = pysqldf(q)
names

,nome_aeroporto,icao_aerodromo,icao_empresa_aerea,razao_social,rotas_saida,rotas_chegada,total_pousos_decolagens
0,Eirunepé Airport (Amaury Feitosa Tomaz Airport),SWEI,PAM,MAP TRANSPORTES AÉREOS LTDA.,2,2,308
1,Palmas–Brigadeiro Lysias Rodrigues Airport,SBPJ,AZU,AZUL LINHAS AÉREAS BRASILEIRAS S/A,6,5,1153
2,Los Angeles International Airport,KLAX,KAL,None,1,1,280
3,Mexico City International Airport,MMMX,AMX,None,1,1,718
4,El Dorado International Airport,SKBO,AVA,None,3,2,572
...,...,...,...,...,...,...,...
252,East Midlands Airport,EGNX,CLX,None,1,1,2
253,Montréal–Pierre Elliott Trudeau International ...,CYUL,ACA,None,1,1,18
254,Afonso Pena International Airport,SBCT,AZU,AZUL LINHAS AÉREAS BRASILEIRAS S/A,18,16,12352
255,Silvio Pettirossi International Airport,SGAS,LAP,None,5,5,410


In [80]:
names.to_csv('airlines_SQL_view2_aerodromos.csv', index=False, encoding='utf-8-sig')